In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xiaoyijie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [97]:
df = pd.read_excel("AdivisenData_REVISE.xlsx")

In [99]:
def rm_number(text):
    return re.sub(r'\d+', '', str(text))

def rm_whitespaces(text):
    return re.sub(r' +', ' ', str(text))

def rm_nonascii(text):
    return re.sub(r'[^\x00-\x7f]', r'', str(text))

def rm_punct(text):
    return re.sub(r'[\"\#\$\%\&\'\(\)\*\+\/\:\;\<\=\>\@\[\\\]\^\_\`\{\|\}\~]', ' ', str(text))
def clean_pipeline_stem(text):
    ps = PorterStemmer()
    #temp = rm_number(text)
    temp = rm_whitespaces(text)
    temp = rm_nonascii(temp)
    temp = rm_punct(temp)
    #STOPWORDS = set(stopwords.words('english'))
    #result = ' '.join(word for word in temp.split() if word not in STOPWORDS) # delete stopwors from text
    #stemming
    result = ' '.join(ps.stem(word) for word in temp.split())
    return result

In [188]:

#feature_cols = ['new','COUNTRY_CODE_revised','CASESTATUS_revised','JURIS_TRIGGER_revised','FINANCIAL_DAMAGES_AMT', 'TOTAL_AMOUNT', 'SETTLEMENT_AMOUNT', 'SETTLEMENT_AMOUNT_CASH','COMPANY_ID']
feature_cols = ['CASE_DESCRIPTION']
X = df[feature_cols]
print(X.head())
label = ['CASE_CATEGORY_revised']
y = df[label]

X_new_stem = X['CASE_DESCRIPTION'].apply(clean_pipeline_stem)


                                    CASE_DESCRIPTION
0  In September 2006, John L. Burgess alleged tha...
1  Greenpoint Credit Corp. initially sued non-Eng...
2  In February 1995 to September 1996, the Wester...
3  Car Dealership Settles Lawsuit over Faxed Ads:...
4  Credit Reporting Company Committed Multiple Vi...


In [11]:
y = y.values.ravel()

In [190]:
X_new_stem_train, X_new_stem_test, y_train, y_test = train_test_split(X_new_stem, y, test_size=0.25, random_state=16, stratify= y)

In [185]:
def clean_pipeline(text):
    ps = PorterStemmer()
    #temp = rm_number(text)
    temp = rm_whitespaces(text)
    temp = rm_nonascii(temp)
    temp = rm_punct(temp)
    return temp
X_new = X['CASE_DESCRIPTION'].apply(clean_pipeline)
X_new_train, X_new_test, y_train, y_test = train_test_split(X_new, y, test_size=0.25, random_state=16, stratify= y)

In [189]:
X_new_stem.head()


0    In septemb 2006, john L. burgess alleg that at...
1    greenpoint credit corp. initi su non-english s...
2    In februari 1995 to septemb 1996, the western ...
3    car dealership settl lawsuit over fax ad A car...
4    credit report compani commit multipl violat A ...
Name: CASE_DESCRIPTION, dtype: object

# <font color=blue>CASE_CATEGORY prediction</font>
***

## predictions using only text info

### without stemming

In [187]:
weights = {0:2000, 1:1}
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(class_weight=weights)),
               ])
logreg.fit(X_new_train, y_train)



y_pred = logreg.predict(X_new_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy 0.9989231664726426
              precision    recall  f1-score   support

           0       0.19      0.33      0.24        18
           1       1.00      1.00      1.00     34342

    accuracy                           1.00     34360
   macro avg       0.60      0.67      0.62     34360
weighted avg       1.00      1.00      1.00     34360



### with stemming

In [191]:
weights = {0:2000, 1:1}
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(class_weight=weights)),
               ])
logreg.fit(X_new_stem_train, y_train)



y_pred = logreg.predict(X_new_stem_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy 0.9989231664726426
              precision    recall  f1-score   support

           0       0.17      0.28      0.21        18
           1       1.00      1.00      1.00     34342

    accuracy                           1.00     34360
   macro avg       0.59      0.64      0.61     34360
weighted avg       1.00      1.00      1.00     34360



##### conclusion: stemming does not contribute to significant improvements
***

### attemptions to find the best class weight ratio, scarce class is weighted heavily 

In [123]:
#fixed model, various weights
weights = {0:2000, 1:1}
print(weights)
logreg6 = Pipeline([('tfidf', TfidfVectorizer(max_features=100)),
                ('clf', LogisticRegression(
            solver='liblinear',class_weight=weights))
               ])
logreg6.fit(X_new_train, y_train)

y_pred = logreg6.predict(X_new_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))


weights2 = {0:3000, 1:1}
print(weights2)
logreg7 = Pipeline([('tfidf', TfidfVectorizer(max_features=100)),
                ('clf', LogisticRegression(
            solver='liblinear',class_weight=weights2))
               ])
logreg7.fit(X_new_train, y_train)

y_pred = logreg7.predict(X_new_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))


weights3 = {0:3000, 1:2}
print(weights3)
logreg8 = Pipeline([('tfidf', TfidfVectorizer(max_features=100)),
                ('clf', LogisticRegression(
            solver='liblinear',class_weight=weights3))
               ])
logreg8.fit(X_new_train, y_train)



y_pred = logreg8.predict(X_new_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

{0: 2000, 1: 1}


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy 0.969441210710128
              precision    recall  f1-score   support

           0       0.01      0.56      0.02        18
           1       1.00      0.97      0.98     34342

    accuracy                           0.97     34360
   macro avg       0.50      0.76      0.50     34360
weighted avg       1.00      0.97      0.98     34360

{0: 3000, 1: 1}


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy 0.9672875436554133
              precision    recall  f1-score   support

           0       0.01      0.56      0.02        18
           1       1.00      0.97      0.98     34342

    accuracy                           0.97     34360
   macro avg       0.50      0.76      0.50     34360
weighted avg       1.00      0.97      0.98     34360

{0: 3000, 1: 2}


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy 0.97430151338766
              precision    recall  f1-score   support

           0       0.01      0.56      0.02        18
           1       1.00      0.97      0.99     34342

    accuracy                           0.97     34360
   macro avg       0.51      0.77      0.50     34360
weighted avg       1.00      0.97      0.99     34360



### The more formal approach to find the best set of parameters should be grid search

In [ ]:
#tried to use grid search and run this cell, but it takes too long, >=20min
from numpy import mean
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
balance = [{0:1000, 1:1}, {0:2000, 1:1}, {0:3000, 1:1}, {0:3000, 1:2}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=36)
# define grid search, here we use the old model in logreg2
grid = GridSearchCV(estimator=logreg2, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X_new, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
from numpy import mean
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
balance = [{0:1000, 1:1}, {0:2000, 1:1}, {0:3000, 1:1}, {0:3000, 1:2}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=6, n_repeats=3, random_state=36)

logreg5 = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(solver='liblinear'))
               ])
# define grid search, here we use the old model in logreg2
grid = GridSearchCV(estimator=logreg5, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X_new, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Predictions using integration of vectorized text info and other features

In [192]:
import numpy as np
other_feature_cols = ['COUNTRY_CODE_revised','CASESTATUS_revised','JURIS_TRIGGER_revised','FINANCIAL_DAMAGES_AMT', 'TOTAL_AMOUNT', 'SETTLEMENT_AMOUNT', 'SETTLEMENT_AMOUNT_CASH','COMPANY_ID']

X_other = df[other_feature_cols]
X_all = pd.concat([X_new,X_other],axis=1,join='inner')
X_all = X_all.fillna(0)
#y_new = y.values.ravel()
y_new = np.array(y_new)

In [194]:
X_all_train, X_all_test, y_train, y_test = train_test_split(X_all, y, test_size=0.25, random_state=16, stratify=y)

In [74]:
from sklearn.base import BaseEstimator, TransformerMixin
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns]

In [103]:
class ShapePrinter(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        print(f'Shape of input: {X.shape}')
        return X

### logistic default solver = 'lbfgs'

In [134]:
from sklearn.pipeline import FeatureUnion
weights0 = {0:3000, 1:2}

logreg8 = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
            ('text', Pipeline([
                ('selector', ColumnSelector(columns='CASE_DESCRIPTION')),
                ('tfidf', TfidfVectorizer(max_features=100)),
                ('shape_printer', ShapePrinter())
            ])),
            ('simple_features', Pipeline([
                ('selector', ColumnSelector(columns=['COUNTRY_CODE_revised','CASESTATUS_revised','JURIS_TRIGGER_revised','FINANCIAL_DAMAGES_AMT', 'TOTAL_AMOUNT', 'SETTLEMENT_AMOUNT', 
                                                     'SETTLEMENT_AMOUNT_CASH','COMPANY_ID'])),
                #('count_vectorizer', CountVectorizer()),
                ('shape_printer', ShapePrinter()) 
            ]))
        ]
    )),
    ('clf', LogisticRegression(class_weight=weights0))
])

print("X_train shape:", X_all_train.shape)
print("y_train shape:", y_train.shape)
logreg8.fit(X_all_train, y_train)



y_pred = logreg8.predict(X_all_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

X_train shape: (103079, 8)
y_train shape: (103079, 1)
Shape of input: (103079, 100)
Shape of input: (103079, 7)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Shape of input: (34360, 100)
Shape of input: (34360, 7)
accuracy 0.995372526193248
              precision    recall  f1-score   support

           0       0.10      1.00      0.18        18
           1       1.00      1.00      1.00     34342

    accuracy                           1.00     34360
   macro avg       0.55      1.00      0.59     34360
weighted avg       1.00      1.00      1.00     34360



### logistic customized solver = 'saga'

In [197]:
# solver = saga
logreg9 = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
            ('text', Pipeline([
                ('selector', ColumnSelector(columns='CASE_DESCRIPTION')),
                ('tfidf', TfidfVectorizer(max_features=100)),
                ('shape_printer', ShapePrinter())
            ])),
            ('simple_features', Pipeline([
                ('selector', ColumnSelector(columns=['COUNTRY_CODE_revised','CASESTATUS_revised','JURIS_TRIGGER_revised','FINANCIAL_DAMAGES_AMT', 'TOTAL_AMOUNT', 'SETTLEMENT_AMOUNT', 
                                                     'SETTLEMENT_AMOUNT_CASH','COMPANY_ID'])),
                #('count_vectorizer', CountVectorizer()),
                ('shape_printer', ShapePrinter()) 
            ]))
        ]
    )),
    ('clf', LogisticRegression(solver='saga',class_weight=weights0))
])

print("X_train shape:", X_all_train.shape)
print("y_train shape:", y_train.shape)
logreg9.fit(X_all_train, y_train)



y_pred = logreg9.predict(X_all_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

X_train shape: (103079, 9)
y_train shape: (103079, 1)
Shape of input: (103079, 100)
Shape of input: (103079, 8)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Shape of input: (34360, 100)
Shape of input: (34360, 8)
accuracy 0.9981955762514552
              precision    recall  f1-score   support

           0       0.23      1.00      0.37        18
           1       1.00      1.00      1.00     34342

    accuracy                           1.00     34360
   macro avg       0.61      1.00      0.68     34360
weighted avg       1.00      1.00      1.00     34360



_word2vec model is too large, so I skip the trial_

# <font color=blue>CASE_TYPE prediction</font>
***

In [198]:
other_feature_cols2 = ['COUNTRY_CODE_revised','CASESTATUS_revised','JURIS_TRIGGER_revised','FINANCIAL_DAMAGES_AMT', 'TOTAL_AMOUNT', 'SETTLEMENT_AMOUNT', 'SETTLEMENT_AMOUNT_CASH', 'AFFECTED_COUNT','COMPANY_ID']

X_other2 = df[other_feature_cols2]
X_all2 = pd.concat([X_new,X_other2],axis=1,join='inner')
X_all2 = X_all2.fillna(0)

In [199]:
y_type = df['CASE_TYPE_revised']
X_train, X_test, y_train, y_test = train_test_split(X_all2, y_type, test_size=0.25, random_state=16, stratify=y_type)

## logistic solver = 'lbfgs'

In [181]:
#weights1 = {"0":92, "1":39, 2:42, 3:1, 4:19, 5:14, 6:72, 7:58, 8:560, 9:29, 10:1890, 11:3022, 12:5, 13:7, 14:2, 15:55}
weights1 = {1:1, 2:39, 3:42, 4:1, 5:19, 6:14, 7:72, 8:58, 9:1, 10:29, 11:1, 12:1, 13:5, 14:7, 15:2, 16:55}
logreg1 = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
            ('text', Pipeline([
                ('selector', ColumnSelector(columns='CASE_DESCRIPTION')),
                ('tfidf', TfidfVectorizer(max_features=1000)),
                ('shape_printer', ShapePrinter())
            ])),
            ('simple_features', Pipeline([
                ('selector', ColumnSelector(columns=['COUNTRY_CODE_revised','CASESTATUS_revised','JURIS_TRIGGER_revised','FINANCIAL_DAMAGES_AMT'
                                                     , 'TOTAL_AMOUNT', 'AFFECTED_COUNT','COMPANY_ID','SETTLEMENT_AMOUNT', 'SETTLEMENT_AMOUNT_CASH'
                                                     ])),
                #('count_vectorizer', CountVectorizer()),
                ('shape_printer', ShapePrinter()) 
            ]))
        ]
    )),
    ('clf', LogisticRegression(class_weight=weights1))
])

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
logreg1.fit(X_train, y_train)



y_pred = logreg1.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

X_train shape: (103079, 10)
y_train shape: (103079,)
Shape of input: (103079, 1000)
Shape of input: (103079, 9)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Shape of input: (34360, 1000)
Shape of input: (34360, 9)
accuracy 0.4474679860302678
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       169
           2       0.00      0.00      0.00       398
           3       0.14      0.02      0.04       368
           4       0.45      1.00      0.62     15166
           5       0.00      0.00      0.00       827
           6       0.43      0.01      0.02      1078
           7       0.01      0.00      0.01       208
           8       0.00      0.00      0.00       259
           9       0.00      0.00      0.00        33
          10       0.00      0.00      0.00       484
          11       0.22      0.44      0.30         9
          12       0.00      0.00      0.00         9
          13       0.00      0.00      0.00      3241
          14       0.00      0.00      0.00      2176
          15       0.38      0.03      0.05      9667
          16       0.00      0.00      0.00       

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## predictions using text info only

In [182]:
#predicts using text info only
logreg2 = Pipeline([('tfidf', TfidfVectorizer(max_features=1000)),
                ('clf', LogisticRegression(
            class_weight=weights1))
               ])
logreg2.fit(X_new_train, y_train)



y_pred = logreg2.predict(X_new_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy 0.14417927823050058


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           1       0.00      0.00      0.00       169
           2       0.01      0.08      0.02       398
           3       0.01      0.08      0.02       368
           4       0.44      0.19      0.27     15166
           5       0.02      0.05      0.03       827
           6       0.03      0.03      0.03      1078
           7       0.01      0.10      0.01       208
           8       0.01      0.09      0.01       259
           9       0.00      0.00      0.00        33
          10       0.02      0.08      0.03       484
          11       0.00      0.00      0.00         9
          12       0.00      0.00      0.00         9
          13       0.09      0.02      0.03      3241
          14       0.06      0.02      0.03      2176
          15       0.28      0.17      0.21      9667
          16       0.01      0.10      0.02       268

    accuracy                           0.14     34360
   macro avg       0.06   

## predictions using customized logistic where solver = 'saga'

In [200]:
#solver = liblinear
logreg1 = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
            ('text', Pipeline([
                ('selector', ColumnSelector(columns='CASE_DESCRIPTION')),
                ('tfidf', TfidfVectorizer(max_features=1000)),
                ('shape_printer', ShapePrinter())
            ])),
            ('simple_features', Pipeline([
                ('selector', ColumnSelector(columns=['COUNTRY_CODE_revised','CASESTATUS_revised','JURIS_TRIGGER_revised','FINANCIAL_DAMAGES_AMT'
                                                     , 'TOTAL_AMOUNT', 'AFFECTED_COUNT','COMPANY_ID','SETTLEMENT_AMOUNT', 'SETTLEMENT_AMOUNT_CASH'
                                                     ])),
                #('count_vectorizer', CountVectorizer()),
                ('shape_printer', ShapePrinter()) 
            ]))
        ]
    )),
    ('clf', LogisticRegression(solver='saga',class_weight=weights1))
])

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
logreg1.fit(X_train, y_train)



y_pred = logreg1.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

X_train shape: (103079, 10)
y_train shape: (103079,)
Shape of input: (103079, 1000)
Shape of input: (103079, 9)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Shape of input: (34360, 1000)
Shape of input: (34360, 9)
accuracy 0.2781140861466822
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       169
           2       0.00      0.00      0.00       398
           3       0.00      0.00      0.00       368
           4       0.00      0.00      0.00     15166
           5       0.00      0.00      0.00       827
           6       0.00      0.00      0.00      1078
           7       0.01      0.00      0.01       208
           8       0.02      0.01      0.01       259
           9       0.00      0.00      0.00        33
          10       0.00      0.00      0.00       484
          11       0.00      0.00      0.00         9
          12       0.00      0.00      0.00         9
          13       0.00      0.00      0.00      3241
          14       0.00      0.00      0.00      2176
          15       0.28      0.99      0.44      9667
          16       0.00      0.00      0.00       

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
